In [12]:
import matplotlib.pyplot as plt
import matplotlib
matplotlib.style.use('ggplot')
import pandas as pd
import numpy as np
from collections import Counter

In [4]:
ba_df = pd.read_csv("AID_1030_datatable_all.csv")

# Load compound info
cs_df = pd.read_csv("AID_1030_compound_smiles.csv",sep='\t',header=0)

# Merge the two
full_df = ba_df.merge(cs_df,on='PUBCHEM_CID')

# Cleanup the compound ID column
full_df["PUBCHEM_CID"] = full_df["PUBCHEM_CID"].astype(int)


/home/joe/anaconda3/envs/pubchem-sklearn/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (0,8,9,12,13,14,15,16,17,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [10]:
outcomes = full_df["PUBCHEM_ACTIVITY_OUTCOME"]
print(outcomes.describe())
print(outcomes.unique())

count       220365
unique           3
top       Inactive
freq        148299
Name: PUBCHEM_ACTIVITY_OUTCOME, dtype: object
['Inconclusive' 'Inactive' 'Active']


In [14]:
# Calculate the frequency of each of the outcomes
c = Counter(outcomes)
active_count = c["Active"]
inactive_count = c["Inactive"]
inconclusive_count = c["Inconclusive"]
print("Active: {}, Inactive: {}, Inconclusive: {}".format(active_count,inactive_count,inconclusive_count))

Active: 16112, Inactive: 148299, Inconclusive: 55954
